In [ ]:
import socket
import time
import threading
from rockcomm.commsocket import CommSocket
from rockcomm.datasave import Data
from rockcomm.connexionthread import ConnexionThread


class DatasendSocket():
    def __init__(self, sock):
        self.sock = sock
        return

    def sendData(self, data):
        #print('sending')
        self.sock.send(data.dtype.encode())
        check = self.sock.recv(2048)
        if check != b'DTYPE RCVED':
            raise RuntimeError('Problem with dtype sending : {}'.format(check))
        #print('dtype sent')
        self.sock.send(str(data.dvalue).encode())
        check = self.sock.recv(2048)
        if check != b'DVALUE RCVED':
            raise RuntimeError('Problem with dtype sending : {}'.format(check))
        #print('dvalue sent')
        self.sock.send('EOD'.encode())
        check = self.sock.recv(2048)
        if check != b'EOD RCVED':
            raise RuntimeError('Problem with EOD sending : {}'.format(check.decode()))
        #print('EOD sent')
        return
        #self.sock.close()

class CommandAnswerSocket(CommSocket):
    def comm(self):
        command = self.sock.recv(2048)
        if command == b'SEND HELLO':
            self.sock.send(b'Hello')
        elif command != b'0':
            self.sock.send(command)
        self.sock.close()




class DatasendThread(threading.Thread):
    def __init__(self, address, data, continuous=True):
        threading.Thread.__init__(self)
        self.address = address
        self.data = data
        self.continuous = continuous
        return
    
    def run(self):
        while(self.continuous):
            print('Datasend thread running new loop')
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.connect(self.address)
            DSendSock = DatasendSocket(self.sock)
            DSendSock.sendData(self.data())
            self.sock.close()
            time.sleep(1)
        print('Datasend Thread at {} shutting down'.format(self.address))
        return

    def stopconnexion(self):
        self.continuous = False

class CommansThread(threading.Thread):
    def __init__(self, address):
        threading.Thread.__init__(self)
        self.address = address
        self.continuous = True
        return
    
    def run(self):
        while(self.continuous):
            print('CommAns thread running new loop')
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.connect(self.address)
            sockCANS = CommandAnswerSocket(self.sock, self.address)
            sockCANS.comm()
            self.sock.close()
            time.sleep(1)
        print('Commans Thread at {} shutting down'.format(self.address))
    
    def stopconnexion(self):
        self.continuous = False

ipaddDSEND = ('localhost', 1111)
ipaddCANS = ('localhost', 1112)
sockDSEND = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sockCANS = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

def timedata():
    return Data('time', time.time())

In [ ]:
CAnsThread = CommansThread(ipaddCANS)
CAnsThread.start()

In [ ]:
#Data('acceleration', 9.806)

DSndThread = DatasendThread(ipaddDSEND, timedata)
DSndThread.start()

In [ ]:
CAnsThread.stopconnexion()

In [ ]:
DSndThread.stopconnexion()